In [2]:
from tkinter import *
from tkinter import filedialog

# A function to allow the user to select the image they wish to analyse. 
# Function inputs args: None. 
# Function output 1: The file path of that which was selected by the user. 
def file_selection_dialog():
    root = Tk()
    root.title('Please select the file in question')
    root.filename = filedialog.askopenfilename(initialdir="/", title="Select A File", filetypes=[("All files", "*.*")])
    file_path = root.filename
    root.destroy()

    return file_path

In [ ]:
from pandas import read_csv 
from sklearn import datasets 
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import numpy as np

def MV_linear_regression(file_path, display_plot, save_plot): 

    ##### (1) Load and prepare data. 
    data =  datasets.load_breast_cancer()
    x, y = data.data, data.target
    
    # Get data dimensions. 
    _, num_features = x.shape
    
    # Split the data into training data and testing data.
    x_training, x_testing, y_training, y_testing = train_test_split(x, y, test_size=0.33, random_state=1234)

    # Scale the data. 
    x_training = StandardScaler().fit_transform(x_training)
    x_testing = StandardScaler().fit_transform(x_testing)
    
    # Convert data to tensors.
    x_training = torch.from_numpy(x_training.astype(np.float32))
    x_testing = torch.from_numpy(x_testing.astype(np.float32))
    
    num_samples = y_training.shape
    y_training = torch.from_numpy(y_training.reshape(num_samples[0], 1))
    y_training = y_training.type(torch.float32)
                                 
    num_samples = y_testing.shape
    y_testing = torch.from_numpy(y_testing.reshape(num_samples[0], 1))
    y_testing = y_testing.type(torch.float32)                                 
   
    ##### (2) Create our model. 
    class LogisticRegression(nn.Module):
        
        def __init__(self, n_input_features):
            super(LogisticRegression, self).__init__()
            self.linear = nn.Linear(n_input_features, 1)
            
        def forward(self, x):
            y_predicted = torch.sigmoid(self.linear(x))
            return y_predicted
    
    # Create an instance of our model. 
    model = LogisticRegression(num_features)
    
    ##### (3) Establish the loss and the optimiser. 
    calc_loss = nn.BCELoss() # Use built in binary cross entropy loss function from PyTorch.
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # We're using stochastic gradient descent. 
    
    ##### (4) Training loop. 
    num_epochs = 10000
    loss_array = []
    for epoch in range(num_epochs):
    
        # Forward pass: compute the output of the layers given the input data
        y_predicted = model(x_training)
        loss = calc_loss(y_predicted, y_training)
        
        # Log the loss per epoch.
        loss_value = loss.detach().numpy()
        loss_value = loss_value.item()
        loss_array.append(loss_value)
        
        # Backward pass: compute the output error with respect to the expected output and then go backward into the network and update the weights using gradient descent.
        loss.backward()
        
        # Update the weights.
        optimizer.step()

        # Zero out the gradients. 
        optimizer.zero_grad()
        
        if epoch % 100 == 0:
            print(f"epoch: {epoch}, loss = {loss.item():.4f}")
    
    with torch.no_grad():
        y_predicted = model(x_testing)
        y_predicted_classes = y_predicted.round()
        accuracy = y_predicted_classes.eq(y_testing).sum().detach().numpy() / float(y_testing.shape[0])
        print(accuracy)